In [1]:
from typing import List

import numpy as np
import scipy.sparse as sp

# Project 4: Spectral clustering users based on their preferences (50 pt)

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
You are given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, you are given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

You are allowed to use the imported functions (`eigsh`, `KMeans`, `normalize`).

## General remarks
Do not add or modify any code outside of the following comment blocks, or where otherwise explicitly stated.

``` python
##########################################################
# YOUR CODE HERE
...
##########################################################
```
After you fill in all the missing code, restart the kernel and re-run all the cells in the notebook.

The following things are **NOT** allowed:
- Using additional `import` statements
- Copying / reusing code from other sources (e.g. code by other students)

If you plagiarise even for a single project task, you won't be eligible for the bonus this semester.

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a _sparse adjacency matrix_ `A` (shape `[N, N]`).
* User preferences are stored in a _feature matrix_ `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [2]:
A = sp.load_npz('A.npz')
F = np.load('F.npy')
categories = np.load('categories.npy', allow_pickle=True).tolist()

In [3]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

In [4]:
print(f'The adjacency matrix is {"symmetric" if (A != A.T).sum() == 0 else "asymmetric"}')

The adjacency matrix is symmetric


# 1. Implementing spectral clustering (35 pt)
## 1.1. Construct the graph Laplacian (10 pt)

First, we need to construct the Laplacian for the given graph (*Do only use sparse operations, see [Scipy Sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)*). 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we will need to use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [5]:
def construct_laplacian(A: sp.csr_matrix, norm_laplacian: bool) -> sp.csr_matrix:
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
    ##########################################################
    # YOUR CODE HERE
    D = A.sum(axis=1).getA1() # getA1() flattened matrix into array
    if not norm_laplacian:
      L = sp.diags(D) - A
    else:
      D[D == 0] = 1 # to avoid division by zero
      sqrt_inv_D = 1.0/np.sqrt(D) # D^{-1/2}
      mat_D = sp.diags(sqrt_inv_D)
      L = sp.eye(A.shape[0]) - mat_D.dot(A.dot(mat_D))
    ##########################################################
    return L

## 1.2. Spectral embedding (10 pt)

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [6]:
from scipy.sparse.linalg import eigsh

In [7]:
help(eigsh)

Help on function eigsh in module scipy.sparse.linalg.eigen.arpack.arpack:

eigsh(A, k=6, M=None, sigma=None, which='LM', v0=None, ncv=None, maxiter=None, tol=0, return_eigenvectors=True, Minv=None, OPinv=None, mode='normal')
    Find k eigenvalues and eigenvectors of the real symmetric square matrix
    or complex hermitian matrix A.
    
    Solves ``A * x[i] = w[i] * x[i]``, the standard eigenvalue problem for
    w[i] eigenvalues with corresponding eigenvectors x[i].
    
    If M is specified, solves ``A * x[i] = w[i] * M * x[i]``, the
    generalized eigenvalue problem for w[i] eigenvalues
    with corresponding eigenvectors x[i].
    
    Parameters
    ----------
    A : ndarray, sparse matrix or LinearOperator
        A square operator representing the operation ``A * x``, where ``A`` is
        real symmetric or complex hermitian. For buckling mode (see below)
        ``A`` must additionally be positive-definite.
    k : int, optional
        The number of eigenvalues and eige

In [8]:
def spectral_embedding(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool) -> np.array:
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    if (A != A.T).sum() != 0:
        raise ValueError("Spectral embedding doesn't work if the adjacency matrix is not symmetric.")
    if num_clusters < 2:
        raise ValueError("The clustering requires at least two clusters.")
    if num_clusters > A.shape[0]:
        raise ValueError(f"We can have at most {A.shape[0]} clusters (number of nodes).")
    ##########################################################
    # YOUR CODE HERE
    L = construct_laplacian(A, norm_laplacian)
    _, eigenvectors = eigsh(L, k=num_clusters, which='SA')
    ##########################################################
    return eigenvectors

## 1.3. Determine the clusters based on the spectral embedding (15 pt)

You should use the K-means algorithm for assigning nodes to clusters, once the spectral embedding is computed.

One thing you should keep in mind, is that when using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm.

In [9]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [10]:
def spectral_clustering(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool, seed: int = 42) -> np.array:
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
    seed : int, default 42
        Random seed to use for the `KMeans` clustering.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    model = KMeans(num_clusters, random_state=seed)
    ##########################################################
    # YOUR CODE HERE
    embeddings = spectral_embedding(A, num_clusters, norm_laplacian)
    if norm_laplacian:
      embeddings = normalize(embeddings, norm='l2', axis=1)
    
    z_pred = model.fit_predict(embeddings)
    ##########################################################
    return z_pred

# 2. Quantitatively evaluate the results (10 pt)

In [11]:
def labels_to_list_of_clusters(z: np.array) -> List[List[int]]:
    """Convert predicted label vector to a list of clusters in the graph.
    This function is already implemented, nothing to do here.
    
    Parameters
    ----------
    z : np.array, shape [N]
        Predicted labels.
        
    Returns
    -------
    list_of_clusters : list of lists
        Each list contains ids of nodes that belong to the same cluster.
        Each node may appear in one and only one partition.
    
    Examples
    --------
    >>> z = np.array([0, 0, 1, 1, 0])
    >>> labels_to_list_of_clusters(z)
    [[0, 1, 4], [2, 3]]
    
    """
    return [np.where(z == c)[0] for c in np.unique(z)]

## 2.1. Compute ratio cut (5 pt)

Your task is to implement functions for computing the **ratio cut** and **normalized cut** for a given partition.

Ratio cut and normalized cut are defined on the slide 14 of the lecture slides.


The function `labels_to_list_of_clusters` can be helpful here.

In [12]:
print(A[1,22], ' ', A[2,22], ' ', A[5, 22], ' ', A[1,3], ' ', A[2,3], ' ', A[5, 3])
print(A[[1,2,5]][:, [22,3]])
print(A[[1,2,5]][:, [22,3]].sum())

3.0   1.0   0.0   1.0   0.0   0.0
  (0, 1)	1.0
  (0, 0)	3.0
  (1, 0)	1.0
5.0


In [13]:
def compute_ratio_cut(A: sp.csr_matrix, z: np.array) -> float:
    """Compute the ratio cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        AdjA[1,10]acency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    ratio_cut : float
        Value of the cut for the given partition of the graph.
        
    """
    
    ##########################################################
    # YOUR CODE HERE
    nodes_clusters = labels_to_list_of_clusters(z)
    N = A.shape[0]
    ratio_cut = 0
    
    for cluster in nodes_clusters:
      nodes_set = set(cluster) # convert to set to make the time complexity for in checks on average O(1)
      not_c = [i for i in range(N) if i not in nodes_set] # V\c_idx
      cut = A[cluster][:, not_c].sum() # sum of inter-cluster edge weights
      ratio_cut += cut / len(cluster)
    ##########################################################
    return ratio_cut

## 2.2. Compute normalized cut (5 pt)

**Important**: if a cluster only contains a single node, define its volume to be 1 to avoid division by zero errors.

In [14]:
def compute_normalized_cut(A: sp.csr_matrix, z: np.array) -> float:
    """Compute the normalized cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    norm_cut : float
        Value of the normalized cut for the given partition of the graph.
        
    """
    
    ##########################################################
    # YOUR CODE HERE
    nodes_clusters = labels_to_list_of_clusters(z)
    N = A.shape[0]
    norm_cut = 0

    for cluster in nodes_clusters:
      nodes_set = set(cluster) # convert to set to make the time complexity for in checks on average O(1)
      not_c = [i for i in range(N) if i not in nodes_set] # V\c_idx
      cut = A[cluster][:, not_c].sum() # sum of inter-cluster edge weights
      vol = A[cluster].sum() # sum degree of nodes in cluster
      norm_cut += cut/vol
    ##########################################################
    return norm_cut

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [15]:
num_clusters = 6

In [16]:
np.random.seed(12903)
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

When using L_unnorm:
 ratio cut = 369.109
 normalized cut = 5.000
 sizes of partitions are: [3379, 1, 1, 1, 1, 1]


In [17]:
np.random.seed(12323)
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 5942.851
 normalized cut = 4.343
 sizes of partitions are: [350, 742, 389, 754, 572, 577]


# 3. Visualize the results (5 pt)

In the final part of the assignment, your task is to print out the 5 most popular types of restaurants visited by the users in each cluster.

In [20]:
def print_top_categories_for_each_cluster(top_k: int, z: np.array, F: sp.csr_matrix, categories: List[str]):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """
    ##########################################################
    # YOUR CODE HERE
    nodes_clusters = labels_to_list_of_clusters(z)
    for c_idx, cluster in enumerate(nodes_clusters):
      preferences = sum(F[cluster])
      top_k_pref = np.argsort(-preferences)[:top_k]
      print('Most popular categories in cluster ', c_idx)
      for cat_idx in top_k_pref:
          print(' -', categories[cat_idx], ',', int(preferences[cat_idx]))
    ##########################################################

In [21]:
np.random.seed(23142)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)

Most popular categories in cluster  0
 - Seafood , 315
 - Mexican , 314
 - Sandwiches , 294
 - Japanese , 291
 - Breakfast & Brunch , 286
Most popular categories in cluster  1
 - Breakfast & Brunch , 636
 - Sandwiches , 528
 - Italian , 514
 - Pizza , 482
 - Coffee & Tea , 473
Most popular categories in cluster  2
 - Specialty Food , 356
 - Thai , 355
 - Breakfast & Brunch , 348
 - Japanese , 333
 - Ethnic Food , 330
Most popular categories in cluster  3
 - Breakfast & Brunch , 664
 - Italian , 626
 - American (Traditional) , 518
 - Sandwiches , 518
 - Pizza , 485
Most popular categories in cluster  4
 - Japanese , 507
 - Breakfast & Brunch , 462
 - Sandwiches , 435
 - Italian , 417
 - Asian Fusion , 414
Most popular categories in cluster  5
 - Japanese , 529
 - Chinese , 441
 - Asian Fusion , 414
 - Sushi Bars , 408
 - Korean , 406


In [5]:
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as pp

import numpy as np
from scipy.linalg import eigh
from scipy.sparse import coo_matrix
from sklearn.cluster import KMeans
import seaborn as sns

sns.set()

def plot_cluster(ebds, epsilon=0.05):
    # Rotate embeddings such that node 4 is always embedded straight down
    four = ebds[3]
    alpha = np.arccos(-four[1] / np.linalg.norm(four))
    c, s = np.cos(alpha), np.sin(alpha)
    R = np.array([[c, -s], [s, c]])
    ebds = ebds @ R

    clusters = KMeans(n_clusters=3).fit_predict(ebds)

    fig, ax = pp.subplots(1, 1, figsize=(6, 6))
    xptp, yptp = ebds.ptp(axis=0) * 0.1
    ax.set_xlim((ebds[:, 0].min() - xptp, ebds[:, 0].max() + xptp))
    ax.set_ylim((ebds[:, 1].min() - yptp, ebds[:, 1].max() + yptp))

    # Disturb points to show nodes that get mapped to the same coordinates
    points = ebds + np.random.rand(*ebds.shape) * epsilon

    colors = ["firebrick", "seagreen", "dodgerblue"]
    bbox_props = dict(boxstyle="circle", alpha=0.5, ec="b", lw=2)
    for i, xyc in enumerate(zip(points, clusters)):
        xy, c = xyc
        bbox_props["fc"] = colors[c]
        pp.text(xy[0], xy[1], str(i + 1), bbox=bbox_props,
                horizontalalignment="center", verticalalignment="center")

    return fig, ax

def laplacian(A):
    D = A.sum(axis=0)
    return np.identity(A.shape[0]) * D - A

edges = np.array([
    (1, 2, 1), (1, 3, 1), (2, 3, 1), (2, 5, 1), (4, 5, 1),
    (4, 6, 1), (5, 6, 2), (6, 9, 1), (7, 8, 3), (7, 9, 1), (8, 9, 1)]).T
edges[:2] -= 1
A = coo_matrix((edges[2], (edges[0], edges[1])), shape=(9, 9)).toarray()
A = A + A.T

_, eig = eigh(laplacian(A), eigvals=(0, 2))
print(eig)
# fig, _ = plot_cluster(eig)
# fig.savefig("problem_3_pre.png")
# 0.31622777
#
A[5, 8] = 40
A[8, 5] = 40
_, eig = eigh(laplacian(A), eigvals=(0, 2))
print(eig)
# fig, _ = plot_cluster(eig)
# fig.savefig("problem_3_post.png")

[[-3.33333333e-01 -4.37607264e-01  2.93891533e-01]
 [-3.33333333e-01 -3.37028446e-01  8.89831963e-02]
 [-3.33333333e-01 -4.37607264e-01  2.93891533e-01]
 [-3.33333333e-01  2.04697370e-15 -5.87783066e-01]
 [-3.33333333e-01 -5.84088369e-02 -3.82874729e-01]
 [-3.33333333e-01  5.84088369e-02 -3.82874729e-01]
 [-3.33333333e-01  4.37607264e-01  2.93891533e-01]
 [-3.33333333e-01  4.37607264e-01  2.93891533e-01]
 [-3.33333333e-01  3.37028446e-01  8.89831963e-02]]
[[-0.33333333 -0.50825814 -0.13718758]
 [-0.33333333 -0.35067024  0.03210615]
 [-0.33333333 -0.50825814 -0.13718758]
 [-0.33333333  0.17520065  0.61051276]
 [-0.33333333  0.07323258  0.33107362]
 [-0.33333333  0.2228469   0.13656023]
 [-0.33333333  0.3330575  -0.47332518]
 [-0.33333333  0.3330575  -0.47332518]
 [-0.33333333  0.2297914   0.11077276]]
